In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.metrics import confusion_matrix
from operator import itemgetter

In [2]:
df = pd.read_csv('interviewchallengesampledata.csv')

In [3]:
df.columns

Index(['Record', 'Image Frame ID', 'Label', 'Score', 'x1', 'y1', 'x2', 'y2',
       'GT Label', 'Threshold', 'GTx1', 'GTx2', 'GTy1', 'GTy2'],
      dtype='object')

In [4]:
df.shape

(3264, 14)

In [5]:
df.head()

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2
0,0,1,alien,0.987645,920.222046,243.421906,962.713257,308.385284,tiger,0.5,926.0,958.0,243.421906,301.0
1,1,2,alien,0.983937,919.544434,255.061249,965.888062,305.472931,tiger,0.5,929.0,958.0,255.061249,302.0
2,2,3,alien,0.984631,919.329346,255.698364,966.233765,305.948425,tiger,0.5,927.0,958.0,255.698364,302.0
3,3,4,alien,0.983412,920.016846,256.790466,966.449585,306.455994,tiger,0.5,928.0,959.0,256.790466,301.0
4,4,5,alien,0.980213,924.729797,227.831360,965.988953,302.414795,tiger,0.5,928.0,958.0,240.000000,302.0


In [6]:
len(pd.unique(df["Record"]))

3264

In [7]:
len(pd.unique(df["Image Frame ID"]))

3264

All values in the `Record` and `Image Frame ID` columns are unique.

In [8]:
pd.unique(df['Label'])

array(['alien', 'baseball', 'gun', 'horse', 'computer', 'tiger', 'boat',
       'lemon', 'moon', 'slenderman', 'elevator', 'puppy'], dtype=object)

In [9]:
pd.unique(df['GT Label'])

array(['tiger', 'alien', nan, 'baseball', 'gun', 'horse', 'computer',
       'puppy', 'elevator', 'moon', 'boat', 'lemon', 'slenderman'],
      dtype=object)

We see `nan` only in the `GT Label` column and not `Label` column.

In [10]:
df_nullGTLabels = df[df['GT Label'].isna()]
df_nullGTLabels

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2
21,21,22,alien,0.992849,964.636108,220.287613,1016.201538,284.434875,NaN,0.5,0.0,0.0,0.0,0.0
34,34,35,baseball,0.814130,1028.764404,124.488708,1101.879395,178.164276,NaN,0.5,0.0,0.0,0.0,0.0
48,48,61,horse,0.139818,890.204773,3.088850,950.131531,56.837898,NaN,0.5,0.0,0.0,0.0,0.0
59,59,72,computer,0.933101,754.241089,118.244843,831.566772,163.777405,NaN,0.5,0.0,0.0,0.0,0.0
60,60,73,computer,0.985782,751.910400,119.423027,835.946289,166.316223,NaN,0.5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,3259,8443,lemon,0.358968,1577.975098,96.424683,1757.348877,166.386780,NaN,0.5,0.0,0.0,0.0,0.0
3260,3260,8503,moon,0.678795,1605.961060,95.251701,1704.947144,233.457184,NaN,0.5,0.0,0.0,0.0,0.0
3261,3261,8601,baseball,0.343997,421.312561,286.117523,545.831970,364.599823,NaN,0.5,0.0,0.0,0.0,0.0
3262,3262,8603,baseball,0.634486,428.349915,289.275024,576.359192,383.126831,NaN,0.5,0.0,0.0,0.0,0.0


In [11]:
np.unique(df_nullGTLabels[["GTx1", "GTx2", "GTy1", "GTy2"]])

array([0.])

Looking at the above rows i.e. objector detector logs, we see that there are almost 1.9k rows where the object detector made a call (not final detection) but there was no GT box annotated in the image. This is because all values in `df_nullGTLabels` are strictly only zeroes. These calls could fall in FALSE POSTIVES (FPs). We will come back to this.

In [12]:
allLabels = list(set(list(pd.unique(df['Label'])) + list(pd.unique(df['GT Label']))))

In [13]:
allLabels = sorted(list(filter(lambda label: isinstance(label, str) , allLabels)))

In [14]:
labelDict = { ix+1:label  for ix, label in enumerate(allLabels) }

In [15]:
labelDict

{1: 'alien',
 2: 'baseball',
 3: 'boat',
 4: 'computer',
 5: 'elevator',
 6: 'gun',
 7: 'horse',
 8: 'lemon',
 9: 'moon',
 10: 'puppy',
 11: 'slenderman',
 12: 'tiger'}

In [16]:
df['Score'].isnull().sum()

0

In [17]:
df['Threshold'].isnull().sum()

0

We see that there zero rows where Score or Threshold columns has null values.

Check how many boxes meet the threshold and how many do not.

In [18]:
df[df['Score'] >= df['Threshold']] # meet the threshold

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2
0,0,1,alien,0.987645,920.222046,243.421906,962.713257,308.385284,tiger,0.5,926.0,958.0,243.421906,301.0
1,1,2,alien,0.983937,919.544434,255.061249,965.888062,305.472931,tiger,0.5,929.0,958.0,255.061249,302.0
2,2,3,alien,0.984631,919.329346,255.698364,966.233765,305.948425,tiger,0.5,927.0,958.0,255.698364,302.0
3,3,4,alien,0.983412,920.016846,256.790466,966.449585,306.455994,tiger,0.5,928.0,959.0,256.790466,301.0
4,4,5,alien,0.980213,924.729797,227.831360,965.988953,302.414795,tiger,0.5,928.0,958.0,240.000000,302.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3254,3254,8323,slenderman,0.532561,1762.785400,98.031258,1829.454102,252.625214,NaN,0.5,0.0,0.0,0.000000,0.0
3256,3256,8329,slenderman,0.515350,1681.483643,81.956169,1763.589111,263.835114,NaN,0.5,0.0,0.0,0.000000,0.0
3258,3258,8345,moon,0.667698,1580.667236,73.267136,1635.032959,219.568436,NaN,0.5,0.0,0.0,0.000000,0.0
3260,3260,8503,moon,0.678795,1605.961060,95.251701,1704.947144,233.457184,NaN,0.5,0.0,0.0,0.000000,0.0


In [19]:
df[df['Score'] < df['Threshold']] # do not meet the threshold

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2
48,48,61,horse,0.139818,890.204773,3.088850,950.131531,56.837898,NaN,0.5,0.0,0.000000,0.0,0.000000
57,57,70,computer,0.466388,787.439270,66.531349,847.361755,127.677879,computer,0.5,789.0,847.361755,73.0,127.677879
73,73,86,tiger,0.462734,812.575439,244.610870,884.141235,312.573090,NaN,0.5,0.0,0.000000,0.0,0.000000
91,91,105,baseball,0.272404,1007.174927,172.022675,1091.642700,224.357452,NaN,0.5,0.0,0.000000,0.0,0.000000
95,95,112,baseball,0.480267,1034.785767,134.195847,1104.828979,190.631302,NaN,0.5,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,3255,8324,slenderman,0.319041,1744.257690,100.154381,1813.766724,277.992340,NaN,0.5,0.0,0.000000,0.0,0.000000
3257,3257,8344,horse,0.254267,1571.641235,86.330704,1613.265991,162.403381,NaN,0.5,0.0,0.000000,0.0,0.000000
3259,3259,8443,lemon,0.358968,1577.975098,96.424683,1757.348877,166.386780,NaN,0.5,0.0,0.000000,0.0,0.000000
3261,3261,8601,baseball,0.343997,421.312561,286.117523,545.831970,364.599823,NaN,0.5,0.0,0.000000,0.0,0.000000


The ones that do not meet the threshold can be categorized as **FALSE NEGATIVES** since the detection box will not appear which would be equivalent to saying that the object detector did not successfully detect the object.

There can be overlap of rows between what we called FALSE POSITIVES (based on `NaN` values under the `GT Label` column) and FALSE NEGATIVES (based on `score < threshold` condition) as shown in the following e.g.

In [20]:
df.loc[48, :] # index 48 is the first row of the previous dataframe

Record                  48
Image Frame ID          61
Label                horse
Score             0.139818
x1                 890.205
y1                 3.08885
x2                 950.132
y2                 56.8379
GT Label               NaN
Threshold              0.5
GTx1                     0
GTx2                     0
GTy1                     0
GTy2                     0
Name: 48, dtype: object

We need to combine multiple conditions to correctly identify the number of FPs, FNs and TPs in the CSV. We still need to calculate the IoU between the GT box and predicted box and see if it meets the IoU threshold. We will assume that this threshold is 50%.

In [21]:
def calculate_iou(row):
    pr_x_topleft, pr_y_topleft, pr_x_bottomright, pr_y_bottomright = row['x1'], row['y1'], row['x2'], row['y2']
    gt_x_topleft, gt_y_topleft, gt_x_bottomright, gt_y_bottomright = row['GTx1'], row['GTy1'], row['GTx2'], row['GTy2']

    if gt_x_topleft == 0 and gt_y_topleft == 0 and gt_x_bottomright == 0 and gt_y_bottomright == 0:
        return 0.0 # this happens when no GT box was annotated
    
    if not pr_x_topleft <= pr_x_bottomright:
        print("pr_x_topleft =", pr_x_topleft)
        print("pr_x_bottomright =", pr_x_bottomright)
        raise Exception("pr_x_topleft <= pr_x_bottomright condition is not being met. Possible issue.")
    if not pr_y_topleft <= pr_y_bottomright:
        print("pr_y_topleft =", pr_y_topleft)
        print("pr_y_bottomright =", pr_y_bottomright)
        raise Exception("pr_y_topleft <= pr_y_bottomright condition is not being met. Possible issue.")

    # since we want even the last pixel along each dimensions to be considered, we 1 to each of the sides
    gt_box_area = (gt_x_bottomright - gt_x_topleft + 1.0) * (gt_y_bottomright - gt_y_topleft + 1.0)
    pr_box_area = (pr_x_bottomright - pr_x_topleft + 1.0) * (pr_y_bottomright - pr_y_topleft + 1.0)

    # calculate coordinates of intersecting rectangle
    # intersecting rectangle should be between gt box and pr_box
    int_x_topleft = max(gt_x_topleft, pr_x_topleft)
    int_y_topleft = max(gt_y_topleft, pr_y_topleft)
    int_x_bottomright = min(gt_x_bottomright, pr_x_bottomright)
    int_y_bottomright = min(gt_y_bottomright, pr_y_bottomright)

    # even if one of the coordinates of bottom-right point of intersection box is less than top-left, 
    # the rectangle has to flip LR or UP to keep shape. But in doing this, it violates bottom-right coordinates >= top-left coordinates
    int_box_area = max(int_x_bottomright - int_x_topleft + 1, 0) * max(int_y_bottomright - int_y_topleft + 1, 0)
    
    iou = int_box_area / (gt_box_area + pr_box_area - int_box_area)
    return iou

In [22]:
df['IoU'] = df.apply(calculate_iou, axis=1)

In [23]:
df # dataframe with new column

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2,IoU
0,0,1,alien,0.987645,920.222046,243.421906,962.713257,308.385284,tiger,0.5,926.0,958.0,243.421906,301.0,0.673821
1,1,2,alien,0.983937,919.544434,255.061249,965.888062,305.472931,tiger,0.5,929.0,958.0,255.061249,302.0,0.590860
2,2,3,alien,0.984631,919.329346,255.698364,966.233765,305.948425,tiger,0.5,927.0,958.0,255.698364,302.0,0.616533
3,3,4,alien,0.983412,920.016846,256.790466,966.449585,306.455994,tiger,0.5,928.0,959.0,256.790466,301.0,0.601990
4,4,5,alien,0.980213,924.729797,227.831360,965.988953,302.414795,tiger,0.5,928.0,958.0,240.000000,302.0,0.611441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,3259,8443,lemon,0.358968,1577.975098,96.424683,1757.348877,166.386780,NaN,0.5,0.0,0.0,0.000000,0.0,0.000000
3260,3260,8503,moon,0.678795,1605.961060,95.251701,1704.947144,233.457184,NaN,0.5,0.0,0.0,0.000000,0.0,0.000000
3261,3261,8601,baseball,0.343997,421.312561,286.117523,545.831970,364.599823,NaN,0.5,0.0,0.0,0.000000,0.0,0.000000
3262,3262,8603,baseball,0.634486,428.349915,289.275024,576.359192,383.126831,NaN,0.5,0.0,0.0,0.000000,0.0,0.000000


In [24]:
df_tp = df[(df['Score'] > df['Threshold']) & (df['IoU'] >= 0.5) & (df['Label'] == df['GT Label'])]

In [25]:
df_tp

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2,IoU
15,15,16,alien,0.982272,937.451111,229.972229,979.905701,295.798462,alien,0.5,951.0,978.000000,240.000000,295.798462,0.547661
23,23,24,alien,0.998562,968.756775,207.904373,1030.661133,271.755310,alien,0.5,989.0,1030.661133,221.000000,271.755310,0.541241
25,25,26,alien,0.996078,983.659546,199.735611,1052.969971,255.341904,alien,0.5,1005.0,1049.000000,208.000000,255.341904,0.546578
26,26,27,alien,0.995432,993.389587,202.199905,1059.060181,255.577408,alien,0.5,1007.0,1053.000000,202.199905,251.000000,0.645616
27,27,28,alien,0.989907,987.749268,196.522629,1058.528076,261.822266,alien,0.5,1005.0,1051.000000,199.000000,255.000000,0.562944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,2160,3359,computer,0.995241,832.654053,245.300217,884.214722,288.656097,computer,0.5,839.0,881.000000,251.000000,286.000000,0.663986
2161,2161,3360,computer,0.992161,820.870056,270.163757,878.062561,311.628540,computer,0.5,834.0,878.062561,270.163757,297.000000,0.507610
2162,2162,3361,computer,0.992996,820.766052,270.103119,882.474426,311.381012,computer,0.5,831.0,878.000000,282.000000,311.381012,0.550053
2164,2164,3363,tiger,0.993573,822.546692,315.660919,880.685486,354.767487,tiger,0.5,831.0,879.000000,317.000000,349.000000,0.681745


In [38]:
df_fp = df[ (df['Score'] > df['Threshold']) & (df['IoU'] >= 0.5) & (df['Label'] != df['GT Label']) \
            | (df['Score'] > df['Threshold']) & (df['IoU'] < 0.5))
        ]

SyntaxError: invalid syntax (<ipython-input-38-f7bc91c34354>, line 2)

In [39]:
df_fp

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2,IoU
0,0,1,alien,0.987645,920.222046,243.421906,962.713257,308.385284,tiger,0.5,926.000000,958.000000,243.421906,301.000000,0.673821
1,1,2,alien,0.983937,919.544434,255.061249,965.888062,305.472931,tiger,0.5,929.000000,958.000000,255.061249,302.000000,0.590860
2,2,3,alien,0.984631,919.329346,255.698364,966.233765,305.948425,tiger,0.5,927.000000,958.000000,255.698364,302.000000,0.616533
3,3,4,alien,0.983412,920.016846,256.790466,966.449585,306.455994,tiger,0.5,928.000000,959.000000,256.790466,301.000000,0.601990
4,4,5,alien,0.980213,924.729797,227.831360,965.988953,302.414795,tiger,0.5,928.000000,958.000000,240.000000,302.000000,0.611441
5,5,6,alien,0.980836,920.885254,257.802155,966.539429,307.847443,tiger,0.5,925.000000,959.000000,257.802155,302.000000,0.664262
6,6,7,alien,0.977441,924.987061,228.362213,965.576294,303.709625,tiger,0.5,929.000000,958.000000,241.000000,301.000000,0.576336
7,7,8,alien,0.976036,919.899658,256.045319,967.320435,306.536530,tiger,0.5,926.000000,959.000000,256.045319,301.000000,0.626677
9,9,10,alien,0.980342,924.209595,229.552673,965.439941,304.308411,tiger,0.5,929.000000,957.000000,241.000000,301.000000,0.552952
10,10,11,alien,0.975542,913.370972,236.199219,954.419312,301.429749,tiger,0.5,927.000000,954.419312,242.000000,301.429749,0.616676


In [28]:
df_fn = df[(df['Score'] < df['Threshold']) & (df['GT Label'].notna()) ] # we use notna() to remove the rows without any GT anno in them

In [29]:
df_fn

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2,IoU
57,57,70,computer,0.466388,787.439270,66.531349,847.361755,127.677879,computer,0.5,789.000000,847.361755,73.000000,127.677879,0.872961
188,188,224,moon,0.148311,779.382812,408.943176,864.128174,568.059509,tiger,0.5,790.000000,858.000000,467.000000,557.000000,0.457345
189,189,226,alien,0.276290,827.821960,463.216309,903.403625,552.438721,alien,0.5,841.000000,898.000000,466.000000,550.000000,0.713523
209,209,246,baseball,0.215476,874.160156,103.797798,952.958618,248.516235,gun,0.5,874.160156,924.000000,112.000000,209.000000,0.428471
255,255,301,gun,0.138319,816.500610,80.625900,878.472534,198.926743,gun,0.5,819.000000,874.000000,97.000000,187.000000,0.678327
452,452,542,lemon,0.322706,979.178040,578.542419,1234.541138,675.739929,computer,0.5,1126.000000,1234.541138,601.000000,675.739929,0.329569
635,635,755,baseball,0.315499,955.829285,884.178467,1116.269775,965.256104,computer,0.5,1390.000000,1116.269775,884.178467,526.000000,0.000000
644,644,775,computer,0.459383,1662.410034,465.422729,1701.128296,497.331055,computer,0.5,1662.410034,1695.000000,465.422729,479.000000,0.374619
646,646,777,computer,0.405028,1681.755127,447.652679,1719.890381,479.155182,computer,0.5,1681.755127,1715.000000,454.000000,479.000000,0.699977
663,663,794,computer,0.246018,1580.822632,429.443573,1623.353882,455.407013,computer,0.5,1580.822632,1615.000000,445.000000,455.407013,0.341868


Let's count the number of rows we will be considering for the confusion matrix.

In [30]:
df_anlsys = pd.concat([ df_tp, df_fp, df_fn]).drop_duplicates()

In [31]:
df_anlsys

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2,IoU
15,15,16,alien,0.982272,937.451111,229.972229,979.905701,295.798462,alien,0.5,951.000000,978.000000,240.000000,295.798462,0.547661
23,23,24,alien,0.998562,968.756775,207.904373,1030.661133,271.755310,alien,0.5,989.000000,1030.661133,221.000000,271.755310,0.541241
25,25,26,alien,0.996078,983.659546,199.735611,1052.969971,255.341904,alien,0.5,1005.000000,1049.000000,208.000000,255.341904,0.546578
26,26,27,alien,0.995432,993.389587,202.199905,1059.060181,255.577408,alien,0.5,1007.000000,1053.000000,202.199905,251.000000,0.645616
27,27,28,alien,0.989907,987.749268,196.522629,1058.528076,261.822266,alien,0.5,1005.000000,1051.000000,199.000000,255.000000,0.562944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,2148,3347,lemon,0.366233,1555.308105,842.318237,1706.568359,916.627197,horse,0.5,1555.308105,945.000000,842.318237,193.000000,0.000000
2149,2149,3348,lemon,0.262943,1553.600586,841.862122,1706.049316,915.551819,horse,0.5,1553.600586,941.000000,841.862122,198.000000,0.000000
2150,2150,3349,horse,0.474786,897.951660,152.409775,942.885010,209.423508,horse,0.5,905.000000,937.000000,152.409775,202.000000,0.626501
2166,2166,3365,lemon,0.410297,1553.046631,843.591553,1710.319336,917.946411,tiger,0.5,1553.046631,887.000000,843.591553,385.000000,0.000000


In [26]:
df.loc[~df_anlsys.index.isin(df.index)]

NameError: name 'df_anlsys' is not defined

In [37]:
df.loc[~df.index.isin(df_anlsys.index)] # rows that are in 

,Record,Image Frame ID,Label,Score,x1,y1,x2,y2,GT Label,Threshold,GTx1,GTx2,GTy1,GTy2,IoU
8,8,9,alien,0.975545,907.360291,234.062027,952.001404,307.278168,tiger,0.5,926.0,952.001404,241.0,301.000000,0.486252
12,12,13,alien,0.951528,909.413330,233.492355,954.278076,302.563202,tiger,0.5,929.0,954.278076,243.0,302.563202,0.495206
16,16,17,alien,0.971789,940.397339,227.018951,987.596802,294.305023,alien,0.5,959.0,987.000000,239.0,294.305023,0.496102
17,17,18,alien,0.981147,940.493591,230.374313,992.547180,301.391785,alien,0.5,963.0,992.547180,240.0,295.000000,0.447720
18,18,19,alien,0.977869,945.236572,231.925201,995.897949,301.559418,alien,0.5,972.0,995.897949,239.0,294.000000,0.382094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,3259,8443,lemon,0.358968,1577.975098,96.424683,1757.348877,166.386780,NaN,0.5,0.0,0.000000,0.0,0.000000,0.000000
3260,3260,8503,moon,0.678795,1605.961060,95.251701,1704.947144,233.457184,NaN,0.5,0.0,0.000000,0.0,0.000000,0.000000
3261,3261,8601,baseball,0.343997,421.312561,286.117523,545.831970,364.599823,NaN,0.5,0.0,0.000000,0.0,0.000000,0.000000
3262,3262,8603,baseball,0.634486,428.349915,289.275024,576.359192,383.126831,NaN,0.5,0.0,0.000000,0.0,0.000000,0.000000
